In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader


from datasets import load_dataset

dataset = load_dataset("cardiffnlp/tweet_eval", "sentiment")

C:\Users\Sushil\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from torch.nn.utils.rnn import pad_sequence
def build_vocab(data):
  vocab = {"<unk>": 0, "<pad>": 1}
  for each_data in data:
    text = each_data['text']
    tokens = text.lower().split()
    for token in tokens:
      if token not in vocab:
        vocab[token] = len(vocab)
  return vocab


vocab = build_vocab(dataset['train'])

vocab_size = len(vocab)


def text_to_ids(text):
  tokens = text.lower().split()
  ids = [vocab.get(token, vocab["<unk>"]) for token in tokens]
  ids = torch.tensor(ids, dtype=torch.long)
  return ids

class SentimentDataset(Dataset):
  def __init__(self, dataset):
    self.dataset = dataset


  def __len__(self):
    return len(self.dataset)


  def __getitem__(self, idx):
    item = self.dataset[idx]
    text = text_to_ids(item['text'])
    label = item['label']
    return text, label


train_dataset = SentimentDataset(dataset['train'])
val_data = SentimentDataset(dataset["validation"])
test_data = SentimentDataset(dataset["test"])


def collate_fn(batch):
  texts, labels = zip(*batch)
  labels = torch.tensor(labels, dtype=torch.long)
  padded_texts = pad_sequence(texts, batch_first=True, padding_value=vocab['<unk>'])
  return padded_texts, labels



train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, collate_fn=collate_fn)



class SentimentLSTM(nn.Module):
  def __init__(self, vocab_size, emb_dim, hidden_dim, output_dim):
    super().__init__()
    self.embeddings = nn.Embedding(vocab_size, emb_dim, padding_idx=vocab['<pad>'])
    self.lstm = nn.LSTM(emb_dim, hidden_dim, batch_first=True)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
    embedded = self.embeddings(x)
    output, (hidden, cell) = self.lstm(embedded)
    output = self.fc(hidden[-1])
    return output


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentimentLSTM(vocab_size, emb_dim=100, hidden_dim=128, output_dim=3).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10

for epoch in range(1, epochs+1):
  model.train()
  total_loss = 0
  for texts, labels in train_loader:
    texts, labels = texts.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(texts)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  print(f"Epoch {epoch}/{epochs} ::: Loss {total_loss/len(train_loader):.4f}")

  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
    for texts, labels in val_loader:
      texts, labels = texts.to(device), labels.to(device)
      outputs = model(texts)
      preds = outputs.argmax(dim=1)
      correct += (preds == labels).sum().item()
      total += labels.size(0)
  print(f"Validation Accuracy: {correct/total:.4f}")






model.eval()
correct = 0
total = 0
with torch.no_grad():
  for texts, labels in test_loader:
    texts, labels = texts.to(device), labels.to(device)
    outputs = model(texts)
    preds = outputs.argmax(dim=1)
    correct += (preds == labels).sum().item()
    total += labels.size(0)
print(f"Test Accuracy: {correct/total:.4f}")



Epoch 1/10 ::: Loss 0.9671
Validation Accuracy: 0.5615
Epoch 2/10 ::: Loss 0.7880
Validation Accuracy: 0.6135
Epoch 3/10 ::: Loss 0.6217
Validation Accuracy: 0.6185
Epoch 4/10 ::: Loss 0.4524
Validation Accuracy: 0.6240
Epoch 5/10 ::: Loss 0.2936
Validation Accuracy: 0.6210
Epoch 6/10 ::: Loss 0.1697
Validation Accuracy: 0.6305
Epoch 7/10 ::: Loss 0.0942
Validation Accuracy: 0.6360
Epoch 8/10 ::: Loss 0.0575
Validation Accuracy: 0.6240
Epoch 9/10 ::: Loss 0.0382
Validation Accuracy: 0.6015
Epoch 10/10 ::: Loss 0.0296
Validation Accuracy: 0.6110
Test Accuracy: 0.5257


In [ ]:
# save the model
# model_path = 'sentiment_llm_bata.pth'
# torch.save(model.state_dict(), model_path)

In [5]:
def predict_sentiment(text):
  model.eval()
  ids = text_to_ids(text)
  ids = ids.to(device)
  with torch.no_grad():
    output = model(ids)
    pred = output.argmax().item() # 0, 1, 2
  label_map = {0: "negative", 1: "neutral", 2: "positive"}
  return label_map[pred]



text = "The icecream was okayish"
predict_sentiment(text)

'neutral'